In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

#parameters
block_size = 64 #size of each context block (sequence length = # of tokens)
batch_size = 128 # number of blocks running in parallel
max_iters = 3000
learning_rate = 3e-3
eval_iters = 100 # show loss data every eval_iters
n_embd = 384 # dimension of each embedding vector
n_head = 4
n_layer = 4
dropout = 0.2

In [ ]:
chars = ""
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)
print(vocab_size)
print(chars)

In [ ]:
# character level tokenizer
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s] # encoder given string returns list of corresponding char integers
decode = lambda l: ''.join([int_to_string[i] for i in l]) # decoder given list of integers returns corresponding string

# examples
print(decode(encode('hello')))

In [ ]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

# splitting to training and validation data
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    # tensor of initial position of x with size batch_size
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y  = x.to(device), y.to(device) # pushing to gpu
    return x,y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets')
print(y)

### Notes
- positional embeddings: not the absolute position of each token in text but rather the relative position in the block-size sequence
    - GPT: we are using learnable embeddings
    - base transformer models: uses sinusoidal fixed embeddings
- using -inf for masking rather than 0
    - if we set the mask to zero, the masked values will not be properly excluded by the softmax function: e^0 = 1 vs. e^(-inf) = 0
    - using -inf to properly zero out the masked positions (ensures the masked positions have no impact on attention scores after softmax)

In [ ]:
class Head(nn.Module):

    def __init__(self, head_size):
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # saving it as model state to avoid repeated computation
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x) # (B, T, hs)
        q = self.query(x) # (B, T, hs)

        # compute attention scores (affinities) 
        # transposing second last and last dimension to allow matrix multiplication, implementing scaling
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**(-0.5) # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        # convert masked values to -inf
        wei = wei.masked_fill(self.tril[:T, :T] ==0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim = -1) # (B, T, T): the -inf gets converted back to 0
        wei = self.dropout(wei)
        v = self.value(x) # (B, T, hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout) # turns a dropout percentage of neurons = 0, prevent overfitting
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        # multiple heads of self-attention in parallel
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd) # even though head_size * num_heads = n_embd adds learnable parameters (eg:bias)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # concatenate each head together along last dimension C: feature dimension
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, C) -> (B, T, [h1, h1, h1, h2, h2, h2, h3, h3, h3...])
        out = self.dropout(self.proj(out))
        return out


In [ ]:
# Block: Decoder
class Block(nn.Module):
    # n_embd: embedding dimension, n_head: the number of heads we'd like
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head # number of features each head will be capturing
        self.sa = MultiHeadAttention(n_head, head_size) # sa: self attention
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    # implementing post norm based on experiment converges better
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(y + x)
        y = self.ffwd(x)
        x = self.ln2(y + x)
        return x

In [ ]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token mapped to a corresponding embedding vector
        # n_embd: we want to hold much larger information for each index
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) # nn.Embedding is learnable parameter
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # decoder blocks running sequentially
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
        # final layer norm that helps model converge better
        self.ln_f = nn.LayerNorm(n_embd)
        # final linear transformation from n_embd -> vocab_size
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights) # recursively applies a given function to every module (layer) within the model

    # initialize weights help training converge better
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets =None):
        # b: batch size, t: sequence length (time), c: vocabulary size
        B, T = index.shape

        # idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arrange(T, device = device)) # (T,C)
        x = tok_emb + pos_emb # (B, T, C): done through torch broadcasting
        x = self.blocks(x) # (B, T, C) feeding through decoder block
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # view: same data as the self tensor but of a different shape
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    # generate new tokens
    def generate(self, index, max_new_tokens):
        #index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            # bigram: focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim = -1)
            # sample from the multinomial distribution
            index_next = torch.multinomial(probs, num_samples = 1) # (B,1)
            # append sampled index to running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

# attempting to generate without training
context = torch.zeros((1,1), dtype = torch.long, device = device) # single next line character
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # switching model in evaluation mode (dropout, batch normalization is turned off)
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out